<a href="https://colab.research.google.com/github/prashant-gulati/colab/blob/main/ReAct_TAO_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReAct Framework: The Thought-Action-Observation Loop

### A Practical Guide to Building AI Agents

---

This tutorial covers the fundamentals of building AI agents using the ReAct (Reasoning + Acting) framework. By the end, you will understand how the Thought-Action-Observation loop works and have a functional agent you can extend for your own use cases.

**Prerequisites:** Basic Python knowledge, an API key from Anthropic or OpenAI

---

## Contents

1. Introduction to ReAct
2. The TAO Loop Explained
3. Setup and Installation
4. Building the Agent
5. Running Examples
6. How It Works
7. Exercises

---
## 1. Introduction to ReAct

ReAct is a framework for building AI agents that was introduced in the paper *"ReAct: Synergizing Reasoning and Acting in Language Models"* by Yao et al. (2022).

### The Problem

Traditional approaches to using LLMs fall into two categories:

- **Reasoning-only approaches** (like Chain-of-Thought): The model thinks step-by-step but cannot interact with external systems
- **Action-only approaches**: The model calls tools directly but lacks the reasoning to know when and why to use them

### The Solution

ReAct combines both. The agent alternates between reasoning about the task and taking actions, using observations from those actions to inform the next step.

| Approach | Can Reason | Can Act | Suitable for Complex Tasks |
|----------|------------|---------|---------------------------|
| Chain-of-Thought | Yes | No | Limited |
| Direct Tool Use | No | Yes | Error-prone |
| ReAct | Yes | Yes | Yes |

---
## 2. The TAO Loop Explained

TAO stands for Thought-Action-Observation. It is the core execution cycle of a ReAct agent.

```
                          User Query
                              |
                              v
                      +-------------+
                +---->|   THOUGHT   |<----+
                |     | (reasoning) |     |
                |     +------+------+     |
                |            |            |
                |            v            |
                |     +-------------+     |
                |     |   ACTION    |     |
                |     | (tool call) |     |
                |     +------+------+     |
                |            |            |
                |            v            |  Repeat until
                |     +-------------+     |  task complete
                +-----| OBSERVATION |-----+
                      | (result)    |
                      +------+------+
                             |
                             v
                      +-------------+
                      |   ANSWER    |
                      +-------------+
```

### Example Trace

Query: *"What is the weather in Paris?"*

```
Thought:  I need to check the weather in Paris. I will use the weather tool.
Action:   get_weather
Input:    {"city": "Paris"}
Observation: Weather in Paris: 64F, Overcast, Humidity: 70%

Thought:  I now have the weather information. I can respond to the user.
Answer:   The weather in Paris is 64F (18C) with overcast skies and 70% humidity.
```

### Components

| Component | Description |
|-----------|-------------|
| Thought | The LLM's reasoning about what to do next |
| Action | The name of the tool to call |
| Action Input | Parameters for the tool, formatted as JSON |
| Observation | The result returned by the tool |
| Final Answer | The complete response to the user |

---
## 3. Setup and Installation

Run the cell below to install the required packages.

In [1]:
!pip install anthropic openai -q

print("Installation complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.9/397.9 kB 6.3 MB/s eta 0:00:00
Installation complete.


In [2]:
import json
import re
from dataclasses import dataclass, field
from typing import Callable, Optional, List, Dict, Any
from getpass import getpass
from IPython.display import display, HTML

print("Imports complete.")

Imports complete.


---
## 4. Building the Agent

We will build the agent in four parts: data structures, tools, the agent class, and visualization.

### 4.1 Data Structures

These classes represent the components of our agent.

In [3]:
@dataclass
class Tool:
    """Represents a tool the agent can use."""
    name: str
    description: str
    func: Callable


@dataclass
class TAOStep:
    """A single Thought-Action-Observation step."""
    thought: str
    action: Optional[str] = None
    action_input: Optional[dict] = None
    observation: Optional[str] = None
    final_answer: Optional[str] = None


@dataclass
class ExecutionTrace:
    """Complete record of an agent execution."""
    query: str
    steps: List[TAOStep] = field(default_factory=list)


print("Data structures defined.")

Data structures defined.


### 4.2 Tools

Tools are functions the agent can call. In a production system, these would connect to real APIs. Here we use simulated responses for demonstration.

In [4]:
def get_weather(city: str) -> str:
    """Get current weather for a city."""
    weather_data = {
        "san francisco": (65, "Partly Cloudy", 72),
        "new york": (78, "Sunny", 55),
        "london": (58, "Rainy", 85),
        "tokyo": (82, "Clear", 60),
        "paris": (64, "Overcast", 70),
        "mumbai": (88, "Humid", 80),
        "sydney": (72, "Sunny", 45),
        "berlin": (55, "Cloudy", 65),
    }
    temp, condition, humidity = weather_data.get(city.lower(), (70, "Clear", 50))
    celsius = round((temp - 32) * 5 / 9)
    return f"Weather in {city}: {temp}F ({celsius}C), {condition}, Humidity: {humidity}%"


def calculator(expression: str) -> str:
    """Evaluate a mathematical expression."""
    allowed_chars = set("0123456789+-*/.() ")
    if not all(c in allowed_chars for c in expression):
        return "Error: Invalid characters in expression."
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return f"{expression} = {result}"
    except Exception as e:
        return f"Error: {str(e)}"


def web_search(query: str) -> str:
    """Search the web for information."""
    return f"""Search results for '{query}':
1. Wikipedia - Comprehensive article about {query}
2. Official Documentation - Technical reference and guides
3. Tutorial - Step-by-step introduction for beginners
4. Research Paper - Academic analysis and findings"""


def get_stock_price(ticker: str) -> str:
    """Get current stock price for a ticker symbol."""
    stock_data = {
        "AAPL": (185.32, 1.2, "Apple Inc."),
        "GOOGL": (142.65, -0.8, "Alphabet Inc."),
        "MSFT": (415.80, 0.5, "Microsoft Corp."),
        "NVDA": (875.40, 2.1, "NVIDIA Corp."),
        "TSLA": (245.20, -1.5, "Tesla Inc."),
        "AMZN": (178.50, 0.9, "Amazon.com Inc."),
        "META": (505.20, 1.8, "Meta Platforms Inc."),
    }
    ticker_upper = ticker.upper()
    if ticker_upper in stock_data:
        price, change, name = stock_data[ticker_upper]
        direction = "+" if change >= 0 else ""
        return f"{ticker_upper} ({name}): ${price:.2f} ({direction}{change}%)"
    return f"{ticker_upper}: Ticker not found."


print("Tools defined.")
print("")
print("Available tools:")
print("  - get_weather(city): Get weather for a city")
print("  - calculator(expression): Evaluate math expressions")
print("  - web_search(query): Search the web")
print("  - get_stock_price(ticker): Get stock prices")

Tools defined.

Available tools:
  - get_weather(city): Get weather for a city
  - calculator(expression): Evaluate math expressions
  - web_search(query): Search the web
  - get_stock_price(ticker): Get stock prices


### 4.3 The ReAct Agent

This is the main agent class that implements the TAO loop.

In [5]:
class ReActAgent:
    """
    A ReAct agent that uses the Thought-Action-Observation loop.
    """

    SYSTEM_PROMPT = """You are a ReAct agent that solves problems using the Thought-Action-Observation loop.

Available Tools:
{tools}

You MUST respond in this EXACT format:

Thought: [Your reasoning about what to do next]
Action: [tool_name]
Action Input: {{"param": "value"}}

OR when you have enough information:

Thought: [Your final reasoning]
Final Answer: [Your complete response to the user]

Rules:
- Always start with Thought
- Action Input must be valid JSON
- Only use tools from the list above
- Provide Final Answer when you have enough information"""

    def __init__(self, api_key: str, provider: str = "anthropic", max_steps: int = 6):
        self.provider = provider
        self.max_steps = max_steps

        if provider == "anthropic":
            import anthropic
            self.client = anthropic.Anthropic(api_key=api_key)
            self.model = "claude-sonnet-4-20250514"
        else:
            import openai
            self.client = openai.OpenAI(api_key=api_key)
            self.model = "gpt-4o"

        self.tools = {
            "get_weather": Tool(
                name="get_weather",
                description='Get weather for a city. Input: {"city": "city name"}',
                func=get_weather
            ),
            "calculator": Tool(
                name="calculator",
                description='Evaluate math. Input: {"expression": "2 + 2"}',
                func=calculator
            ),
            "web_search": Tool(
                name="web_search",
                description='Search the web. Input: {"query": "search terms"}',
                func=web_search
            ),
            "get_stock_price": Tool(
                name="get_stock_price",
                description='Get stock price. Input: {"ticker": "AAPL"}',
                func=get_stock_price
            ),
        }

    def _get_tools_description(self) -> str:
        return "\n".join(
            f"- {tool.name}: {tool.description}"
            for tool in self.tools.values()
        )

    def _call_llm(self, messages: List[Dict]) -> str:
        system_prompt = self.SYSTEM_PROMPT.format(tools=self._get_tools_description())

        if self.provider == "anthropic":
            response = self.client.messages.create(
                model=self.model,
                max_tokens=1024,
                system=system_prompt,
                messages=messages
            )
            return response.content[0].text
        else:
            response = self.client.chat.completions.create(
                model=self.model,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    *messages
                ]
            )
            return response.choices[0].message.content

    def _parse_response(self, text: str) -> TAOStep:
        step = TAOStep(thought="")

        thought_match = re.search(
            r"Thought:\s*(.+?)(?=\n(?:Action|Final Answer)|\Z)",
            text,
            re.DOTALL
        )
        if thought_match:
            step.thought = thought_match.group(1).strip()

        final_match = re.search(r"Final Answer:\s*(.+)", text, re.DOTALL)
        if final_match:
            step.final_answer = final_match.group(1).strip()
            return step

        action_match = re.search(r"Action:\s*(\w+)", text)
        if action_match:
            step.action = action_match.group(1)

        input_match = re.search(r"Action Input:\s*(\{.*?\})", text, re.DOTALL)
        if input_match:
            try:
                step.action_input = json.loads(input_match.group(1))
            except json.JSONDecodeError:
                step.action_input = {}

        return step

    def _execute_tool(self, action: str, inputs: Dict) -> str:
        if action not in self.tools:
            return f"Error: Unknown tool '{action}'"
        try:
            return self.tools[action].func(**(inputs or {}))
        except Exception as e:
            return f"Error: {e}"

    def run(self, query: str, verbose: bool = True) -> ExecutionTrace:
        trace = ExecutionTrace(query=query)
        messages = [{"role": "user", "content": f"Query: {query}"}]

        if verbose:
            print("")
            print("=" * 70)
            print(f"Query: {query}")
            print("=" * 70)

        for step_num in range(1, self.max_steps + 1):
            response_text = self._call_llm(messages)
            step = self._parse_response(response_text)

            if verbose:
                print(f"")
                print(f"Step {step_num}")
                print("-" * 40)
                print(f"Thought: {step.thought}")

            if step.final_answer:
                if verbose:
                    print(f"")
                    print(f"Final Answer: {step.final_answer}")
                trace.steps.append(step)
                break

            if step.action:
                if verbose:
                    print(f"Action: {step.action}")
                    print(f"Input: {json.dumps(step.action_input)}")

                observation = self._execute_tool(step.action, step.action_input)
                step.observation = observation

                if verbose:
                    print(f"Observation: {observation}")

                messages.append({"role": "assistant", "content": response_text})
                messages.append({"role": "user", "content": f"Observation: {observation}"})

            trace.steps.append(step)

        return trace


print("ReActAgent class defined.")

ReActAgent class defined.


### 4.4 Visualization

A function to display the execution trace in a clean format.

In [6]:
def visualize_trace(trace: ExecutionTrace):
    """Display the execution trace in a formatted view."""

    html = """
    <style>
        .tao-container {
            font-family: -apple-system, BlinkMacSystemFont, 'SF Pro Text', 'Segoe UI', Roboto, Helvetica, Arial, sans-serif;
            max-width: 720px;
            margin: 24px auto;
            color: #1d1d1f;
            font-size: 14px;
            line-height: 1.5;
        }
        .tao-header {
            background: #fbfbfd;
            padding: 20px 24px;
            border: 1px solid #d2d2d7;
            border-bottom: none;
            border-radius: 12px 12px 0 0;
        }
        .tao-header h3 {
            margin: 0 0 6px 0;
            font-size: 15px;
            font-weight: 600;
            letter-spacing: -0.01em;
            color: #1d1d1f;
        }
        .tao-header p {
            margin: 0;
            color: #6e6e73;
            font-size: 13px;
        }
        .tao-step {
            background: #ffffff;
            padding: 20px 24px;
            border-left: 1px solid #d2d2d7;
            border-right: 1px solid #d2d2d7;
            border-bottom: 1px solid #e8e8ed;
        }
        .step-number {
            font-size: 11px;
            font-weight: 600;
            color: #86868b;
            text-transform: uppercase;
            letter-spacing: 0.5px;
            margin-bottom: 14px;
        }
        .tao-block {
            margin: 10px 0;
            padding: 12px 14px;
            border-radius: 6px;
            font-size: 13px;
        }
        .thought-block {
            background: #f5f5f7;
            border-left: 2px solid #86868b;
        }
        .action-block {
            background: #fffbeb;
            border-left: 2px solid #ca8a04;
        }
        .observation-block {
            background: #f0fdf4;
            border-left: 2px solid #16a34a;
        }
        .answer-block {
            background: #eff6ff;
            border-left: 2px solid #2563eb;
        }
        .block-label {
            font-size: 10px;
            font-weight: 600;
            color: #6e6e73;
            margin-bottom: 4px;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }
        .block-content {
            color: #1d1d1f;
        }
        code {
            font-family: 'SF Mono', Monaco, Consolas, monospace;
            font-size: 12px;
            background: rgba(0, 0, 0, 0.05);
            padding: 1px 5px;
            border-radius: 3px;
        }
        .tao-footer {
            background: #fbfbfd;
            padding: 12px 24px;
            border: 1px solid #d2d2d7;
            border-top: none;
            border-radius: 0 0 12px 12px;
            text-align: center;
            font-size: 12px;
            color: #86868b;
        }
    </style>
    """

    html += f"""
    <div class="tao-container">
        <div class="tao-header">
            <h3>Execution Trace</h3>
            <p>{trace.query}</p>
        </div>
    """

    for i, step in enumerate(trace.steps, 1):
        html += f"""
        <div class="tao-step">
            <div class="step-number">Step {i}</div>

            <div class="tao-block thought-block">
                <div class="block-label">Thought</div>
                <div class="block-content">{step.thought}</div>
            </div>
        """

        if step.action:
            html += f"""
            <div class="tao-block action-block">
                <div class="block-label">Action</div>
                <div class="block-content">
                    <code>{step.action}</code> with input <code>{json.dumps(step.action_input)}</code>
                </div>
            </div>

            <div class="tao-block observation-block">
                <div class="block-label">Observation</div>
                <div class="block-content">{step.observation}</div>
            </div>
            """

        if step.final_answer:
            html += f"""
            <div class="tao-block answer-block">
                <div class="block-label">Final Answer</div>
                <div class="block-content">{step.final_answer}</div>
            </div>
            """

        html += "</div>"

    html += f"""
        <div class="tao-footer">
            Completed in {len(trace.steps)} step(s)
        </div>
    </div>
    """

    display(HTML(html))


print("Visualization function defined.")

Visualization function defined.


---
## 5. Running Examples

First, configure your API key.

In [7]:
print("API Configuration")
print("=" * 40)
print("")
print("Select provider:")
print("  1. Anthropic (Claude)")
print("  2. OpenAI (GPT-4)")
print("")

choice = input("Enter 1 or 2: ").strip()
PROVIDER = "anthropic" if choice == "1" else "openai"

provider_name = "Anthropic" if PROVIDER == "anthropic" else "OpenAI"
print(f"")
print(f"Enter your {provider_name} API key:")
API_KEY = getpass("API Key: ")

print(f"")
print(f"Provider: {provider_name}")
print("API key configured.")

API Configuration

Select provider:
  1. Anthropic (Claude)
  2. OpenAI (GPT-4)

Enter 1 or 2: 2

Enter your OpenAI API key:
API Key: ··········

Provider: OpenAI
API key configured.


In [8]:
# Create the agent
agent = ReActAgent(api_key=API_KEY, provider=PROVIDER)
print("Agent initialized.")

Agent initialized.


### Example 1: Weather Query

In [9]:
trace1 = agent.run("What is the weather in Tokyo? What should I wear outside?")
visualize_trace(trace1)


Query: What is the weather in Tokyo? What should I wear outside?

Step 1
----------------------------------------
Thought: To provide suitable clothing advice, I need to know the current weather conditions in Tokyo.
Action: get_weather
Input: {"city": "Tokyo"}
Observation: Weather in Tokyo: 82F (28C), Clear, Humidity: 60%

Step 2
----------------------------------------
Thought: With the current temperature at 82F (28C) and clear skies in Tokyo, it's quite warm and likely sunny. Light clothing such as a t-shirt and shorts would be comfortable. Don't forget sunglasses and sunscreen for sun protection.

Final Answer: The weather in Tokyo is warm and clear with a temperature of 82F (28C) and 60% humidity. You should wear light clothing like a t-shirt and shorts, and consider using sunglasses and sunscreen for sun protection.


### Example 2: Calculation

In [10]:
trace2 = agent.run("Calculate 1234 multiplied by 5678.")
visualize_trace(trace2)


Query: Calculate 1234 multiplied by 5678.

Step 1
----------------------------------------
Thought: I need to calculate the product of 1234 and 5678 to provide the correct answer.
Action: calculator
Input: {"expression": "1234 * 5678"}
Observation: 1234 * 5678 = 7006652

Step 2
----------------------------------------
Thought: I have calculated the product of 1234 and 5678.

Final Answer: 1234 multiplied by 5678 equals 7,006,652.


### Example 3: Information Search

In [11]:
trace3 = agent.run("Search for information about transformer architecture.")
visualize_trace(trace3)


Query: Search for information about transformer architecture.

Step 1
----------------------------------------
Thought: I need to search the web to find the most current information about transformer architecture.
Action: web_search
Input: {"query": "transformer architecture"}
Observation: Search results for 'transformer architecture':
1. Wikipedia - Comprehensive article about transformer architecture
2. Official Documentation - Technical reference and guides
3. Tutorial - Step-by-step introduction for beginners
4. Research Paper - Academic analysis and findings

Step 2
----------------------------------------
Thought: There are several sources of information about transformer architecture. Wikipedia provides a comprehensive overview, the official documentation contains technical details, there is a tutorial for beginners, and a research paper that offers an academic analysis. I can summarize the general concept of transformer architecture.

Final Answer: Transformer architecture is 

### Example 4: Stock Price

In [12]:
trace4 = agent.run("What is the current stock price of NVIDIA?")
visualize_trace(trace4)


Query: What is the current stock price of NVIDIA?

Step 1
----------------------------------------
Thought: To find the current stock price of NVIDIA, I need to look up its ticker symbol first, and then fetch the stock price using that symbol.
Action: web_search
Input: {"query": "NVIDIA stock ticker symbol"}
Observation: Search results for 'NVIDIA stock ticker symbol':
1. Wikipedia - Comprehensive article about NVIDIA stock ticker symbol
2. Official Documentation - Technical reference and guides
3. Tutorial - Step-by-step introduction for beginners
4. Research Paper - Academic analysis and findings

Step 2
----------------------------------------
Thought: The Nvidia stock ticker symbol is a common knowledge and it is "NVDA". Now I can get the current stock price for NVIDIA using its ticker symbol.
Action: get_stock_price
Input: {"ticker": "NVDA"}
Observation: NVDA (NVIDIA Corp.): $875.40 (+2.1%)

Step 3
----------------------------------------
Thought: I have obtained the current stoc

### Custom Query

Enter your own query below.

In [13]:
custom_query = input("Enter your query: ")

if custom_query.strip():
    custom_trace = agent.run(custom_query)
    visualize_trace(custom_trace)

Enter your query: What is the current stock price of NVIDIA?

Query: What is the current stock price of NVIDIA?

Step 1
----------------------------------------
Thought: I need to get the latest stock price for NVIDIA, and for that, I need to find the appropriate ticker symbol for NVIDIA to use with the stock price tool.
Action: web_search
Input: {"query": "NVIDIA stock ticker symbol"}
Observation: Search results for 'NVIDIA stock ticker symbol':
1. Wikipedia - Comprehensive article about NVIDIA stock ticker symbol
2. Official Documentation - Technical reference and guides
3. Tutorial - Step-by-step introduction for beginners
4. Research Paper - Academic analysis and findings

Step 2
----------------------------------------
Thought: The search results suggest that the Wikipedia article likely contains the required information about NVIDIA's stock ticker symbol. I will use that to find out.
Action: web_search
Input: {"query": "NVIDIA ticker symbol"}
Observation: Search results for 'NVID

---
## 6. How It Works

When you run a query, the following sequence occurs:

1. **Query submitted** — Your question is sent to the LLM with instructions about available tools

2. **LLM generates Thought** — The model reasons about what it needs to do

3. **LLM selects Action** — Based on its reasoning, it chooses a tool and provides parameters

4. **Tool executes** — Our code runs the actual function and captures the result

5. **Observation returned** — The result is sent back to the LLM

6. **Loop continues** — Steps 2–5 repeat until the LLM decides it has enough information

7. **Final Answer** — The LLM provides a complete response to the user

### Comparison

| Without ReAct | With ReAct |
|---------------|------------|
| LLM might hallucinate data | Data comes from actual tool calls |
| No access to current information | Can query live APIs |
| Cannot perform calculations reliably | Uses calculator for accuracy |
| Reasoning is opaque | Reasoning is explicit and observable |

---
## 7. Exercises

### Exercise 1: Add a New Tool

Implement a tool that returns the current date and time.

In [14]:
from datetime import datetime

def get_current_time(timezone: str = "UTC") -> str:
    """Returns the current date and time."""
    now = datetime.now()
    return f"Current time: {now.strftime('%Y-%m-%d %H:%M:%S')}"

# Test
print(get_current_time())

Current time: 2026-02-01 10:35:48


### Exercise 2: Multi-Step Query

Try a query that requires multiple tool calls.

In [15]:
multi_trace = agent.run(
    "What is the weather in London, and what is the temperature in Celsius multiplied by 2?"
)
visualize_trace(multi_trace)


Query: What is the weather in London, and what is the temperature in Celsius multiplied by 2?

Step 1
----------------------------------------
Thought: I need to first find out the current weather and temperature in London. Then, I'll calculate the temperature in Celsius multiplied by 2.
Action: get_weather
Input: {"city": "London"}
Observation: Weather in London: 58F (14C), Rainy, Humidity: 85%

Step 2
----------------------------------------
Thought: I have the current temperature in London, which is 14 degrees Celsius. I need to multiply this temperature by 2 to get the final result.
Action: calculator
Input: {"expression": "14 * 2"}
Observation: 14 * 2 = 28

Step 3
----------------------------------------
Thought: I have calculated the temperature in Celsius multiplied by 2.

Final Answer: The current weather in London is rainy with 85% humidity, and the temperature is 14°C. When multiplied by 2, the temperature becomes 28°C.


---
## Summary

This tutorial covered the ReAct framework and its core mechanism, the Thought-Action-Observation loop. You built a working agent that can reason about tasks, call tools, and synthesize results into coherent answers.

The key insight of ReAct is that by interleaving reasoning and acting, agents become more capable and their behavior becomes more interpretable.

### References

- Yao et al. (2022). *ReAct: Synergizing Reasoning and Acting in Language Models.* arXiv:2210.03629
- Anthropic Documentation: Tool Use
- LangChain: ReAct Agent Implementation

---